In [13]:
import pandas as pd
import numpy as np
import numbers, sklearn
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import svm, decomposition, preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc


In [14]:
TRAIN, TEST = 0, 1

print(sklearn.__version__)

0.17


In [15]:
print('Reading CSV Files...')
train = pd.read_csv('train_users_2.csv', header=0)
test = pd.read_csv('test_users.csv')
filled = [train.copy(), test.copy()]

Reading CSV Files...


In [20]:
print('Cleaning Data: Converting all entries to numerical type, filling NaNs, etc...')
le = preprocessing.LabelEncoder()
for i, df in enumerate(filled):
    for feature in [col for col in df.columns if col not in ['id', 'country_destination']]: #Excluding id
        if df[feature].dtype not in [int, float]:
            if 'date' in feature:
                df[feature] = pd.to_datetime(df[feature])
                df[feature] = (df[feature] - df[feature].min()) / np.timedelta64(1,'D')
            else:
                mapping = {value: i for i, value in enumerate([_ for _ in df[feature].unique() if _ is not np.nan])}
                mapping_full = lambda x: mapping[x] if x not in ['-unknown-', np.nan] else np.nan
                df[feature] = df[feature].map(mapping_full)
                #print(df[feature])
                #df[feature] = le.fit_transform(df[feature])
        else:
            df[feature].fillna(df[feature].mean())
    #df = df.fillna(-1)
    print(df)

Cleaning Data: Converting all entries to numerical type, filling NaNs, etc...
                id  date_account_created  timestamp_first_active  gender  age  \
0       gxn3p5htnn                   178                       0     NaN  NaN   
1       820tgsjxq7                   509                       1       1   38   
2       4ft3gnwmtx                   270                       2       2   56   
3       bjjt8pjhuk                   703                       3       2   42   
4       87mebub9p4                   256                       4     NaN   41   
5       osr2jwljor                     0                       5     NaN  NaN   
6       lsw9q7uk0j                     1                       6       2   46   
7       0d01nltbrs                     2                       7       2   47   
8       a1vcnhxeij                     3                       8       2   50   
9       6uh8zyj2gn                     3                       9     NaN   46   
10      yuuqmid2rp             

In [17]:
print('Dropping columns of data entirely missing on at least one set...')
for col in filled[TEST].columns:
    if any(df[col].isnull().all() for df in filled):
        for i, df in enumerate(filled):
            filled[i] = filled[i].drop(col, axis=1)

Dropping columns of data entirely missing on at least one set...


In [18]:
print('Isolating train/test sets...')
X_train, y_train = filled[TRAIN].ix[:, 1:-1], filled[TRAIN].ix[:, -1]
X_test = filled[TEST].ix[:, 1:]

Isolating train/test sets...


In [19]:
#model = LogisticRegression()
#model = Classifier(
#    layers=[
#        Layer("Rectifier", units=100),
#        Layer("Linear")],
#    learning_rate=0.02,
#    n_iter=10)
model = RandomForestClassifier(n_estimators=20)
model.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

print('Analyzing Logistic Regression...')
Disbursed_lg = model.predict_proba(X_test)
fpr, tpr, _ = roc_curve(model.predict(X_test), Disbursed_lg[:,1])
roc_auc = auc(fpr, tpr)
print(roc_auc)

In [ ]:
print('Making prediction...')
y_test = pd.DataFrame(model.predict(X_test))
to_output = pd.concat([filled[TEST]['id'], y_test], axis=1)

In [ ]:
print('Writing prediction to submission file...')
to_output.to_csv('submission.csv', index=False, header=['id', 'country'])